In [6]:

from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd

In [7]:
!wget -nc https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/43708c23-2b77-48fd-9986-fa97691a2d59/download/covid19_tia_zonas_basicas_salud_s.csv 2> /dev/null

In [10]:
df = pd.read_csv("covid19_tia_zonas_basicas_salud_s.csv", sep=";", encoding='latin-1')
df.fecha_informe = pd.to_datetime(df.fecha_informe)
df.tasa_incidencia_acumulada_ultimos_14dias = pd.to_numeric(df.tasa_incidencia_acumulada_ultimos_14dias.str.replace(",", "."))

In [12]:
plt.figure(1, title='Serie temporal de la tasa de incidencia acumulada a 14 días')

for zbs, partdf in df.groupby("zona_basica_salud"):
    partdf = partdf.sort_values("fecha_informe")
    x = partdf["fecha_informe"]
    y = partdf["tasa_incidencia_acumulada_ultimos_14dias"]
    plt.plot(x, y, label=zbs)
plt.show()